In [0]:
#Hello
import pandas as pd
import string
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
df = pd.read_csv('train.csv', index_col="Loan_ID")

In [3]:
df.head()

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
Loan_ID,,,,,,,,,,,,,
10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [4]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

#### Handling missing data

In [0]:
def cleaning_data(df):
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].str.replace(",","")
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].astype(float)
    df["Length_Employed"] = df["Length_Employed"].str.replace(r'[><+]+',"")
    df["Length_Employed"] = df["Length_Employed"].str.extract('(\d+)').astype(float)
    df["Length_Employed"] = df["Length_Employed"].transform(lambda x: x.fillna(-1))
    df["Annual_Income"] = df.groupby(["Length_Employed"])["Annual_Income"].transform(lambda x: x.fillna(x.median()))
    df["Home_Owner"] = df["Home_Owner"].transform(lambda x: x.fillna("NA"))
    df["Months_Since_Deliquency"] = df["Months_Since_Deliquency"].transform(lambda x: x.fillna(-1))
    return df

In [0]:
df = cleaning_data(df)

In [0]:
def label_encode(df):
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    df["Income_Verified"] = le.fit_transform(df["Income_Verified"])
    df["Home_Owner"] = le.fit_transform(df["Home_Owner"])
    df["Purpose_Of_Loan"] = le.fit_transform(df["Purpose_Of_Loan"])
    df["Gender"] = le.fit_transform(df["Gender"])
    return df

In [0]:
df = label_encode(df)

In [0]:
original_df = df.copy()
df.to_csv("clean_export.csv")
df = df.drop("Interest_Rate", axis=1)

In [0]:
def normalize_data(df):
  mapper = DataFrameMapper([(df.columns,MinMaxScaler())])
  scaled_features = mapper.fit_transform(df.copy(), 4)
  scaled_features_df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
  return scaled_features_df

In [0]:
scaled_features_df = normalize_data(df)

In [0]:
X_features = scaled_features_df

In [0]:
y = original_df["Interest_Rate"]

In [14]:
y

Loan_ID
10000001    1
10000002    3
10000003    3
10000004    3
10000005    1
           ..
10164305    2
10164306    2
10164307    1
10164308    3
10164309    2
Name: Interest_Rate, Length: 164309, dtype: int64

In [0]:
#instantiate model and train
# model = XGBClassifier(learning_rate = 0.05, n_estimators=500, max_depth=5)
# model.fit(X_features, y)

In [0]:
## Rohit Random Forest
n_estimators = [int(x) for x in range(200, 400, 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in range(100, 500, 10)]
max_depth.append(None)

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth
              }

In [0]:
# rf = RandomForestClassifier()
# # Random search of parameters, using default 5 fold cross validation, 
# # search across different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid)
# # Fit the random search model
# rf_random.fit(X_features, y)

In [0]:
#rf_random.best_params_

In [0]:
n_estimators = 500
max_depth = 100
max_features = "auto"


In [0]:
def rforestModel_train():
    n_estimators = rf_random.best_params_["n_estimators"]
    max_depth = rf_random.best_params_["max_depth"]
    max_features = rf_random.best_params_["max_features"]
    rf_Model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, 
                                    max_features = max_features)
    #rf_Model.fit(X_features, y)
    return rf_Model

In [0]:
#rf_Model = rforestModel_train()

In [0]:
ada_boost = AdaBoostClassifier(n_estimators=500, random_state=12)
XGB_boost = XGBClassifier(learning_rate = 0.05, n_estimators=500, max_depth=5)
rf_Model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, 
                                    max_features = max_features)
LogReg_Model = LogisticRegression(multi_class = 'multinomial')
 
voting_clf = VotingClassifier(estimators=[('XGB_boost', XGB_boost), ('ada_boost', ada_boost), ('randomForest', rf_Model),('LogReg_Model',LogReg_Model)], voting='hard')
voting_clf.fit(X_features, y)

In [0]:
test_df = pd.read_csv("test.csv", index_col = "Loan_ID")

In [0]:
test_df.isna().sum()

In [0]:
test_df = cleaning_data(test_df)

In [0]:
test_df = label_encode(test_df)

In [0]:
scaled_features_test_df = normalize_data(test_df)

In [0]:
predictions = voting_clf.predict(scaled_features_test_df)

In [0]:
# xgBoostpredictions = model.predict(scaled_features_test_df)

In [0]:
# rfpredictions = rf_Model.predict(scaled_features_test_df)

In [0]:
output_df = pd.DataFrame({'Loan_ID': scaled_features_test_df.index, 'Interest_Rate': predictions})

In [0]:
output_df.to_csv("solution.csv", index=False)